In [ ]:
import json
import transformers
import time
import logging
import os
import wandb
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import datasets
import numpy as np
from datasets import load_dataset, load_metric
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from fly_src.models.modeling_roberta import RobertaForSequenceClassification
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from train_utils import *


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

def hook_fn(module, grad_input, grad_output):
    print(f"{module} has dW {grad_input[1]} and scaler value {module.scaler}")
    
    
def factor_balance(mid_blksz, out_blksz):
    total = mid_blksz * out_blksz

# @Wenxuan
# Use a diagonal matrix or a scaler to scale output of monarch factors
class Scaler(nn.Module):
    def __init__(self, out_features):
        super().__init__()
        self.scaler = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        x = self.scaler * x
        x = F.layer_norm(x, x.shape[1:])
        # layernorm to avoid vanishing gradient
        return x

x = torch.ones(100, 100)
y = torch.full((100, 100), 2)
model = Scaler(100)
model.register_backward_hook(hook_fn)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
for i in range(100):
    loss = F.mse_loss(model(x), y)
    loss.backward()
    optimizer.step()
    